In [9]:
import numpy as np
import scipy
from activ import load_data
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from activ.data_normalization import data_normalization
from scipy.spatial.distance import pdist
from sklearn.model_selection import KFold
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
import h5py

In [12]:
rawdata = load_data(uoinmf=False)
raw_bm = rawdata.biomarkers
raw_oc = rawdata.outcomes

knum = 10
kf = KFold(n_splits=knum)

In [14]:
for cv, (train_index, test_index) in enumerate(kf.split(raw_bm)):
    train, test = raw_bm[train_index], raw_bm[test_index]
    train_bm = train
    test_bm = test
    
    nmf = NMF(beta_loss='kullback-leibler', solver='mu', max_iter=400, init='random')
    W = nmf.fit_transform(train_bm)
    H = nmf.components_
    k = H.shape[0]
    print(W.shape, H.shape)
    mse = mean_squared_error(train_bm, W@H)
    print(k,mse)

(222, 235) (235, 235)
235 0.005696885230670956
(222, 235) (235, 235)
235 0.005978778519043715
(222, 235) (235, 235)
235 0.00659917022265417
(222, 235) (235, 235)
235 0.006362723085176256
(222, 235) (235, 235)
235 0.005750333691893099
(222, 235) (235, 235)
235 0.005798241831244305
(222, 235) (235, 235)
235 0.00540097478006858
(223, 235) (235, 235)
235 0.006215473075091339
(223, 235) (235, 235)
235 0.005182909943995049
(223, 235) (235, 235)
235 0.0056812910078334125
